Source for this https://blog.paperspace.com/writing-resnet-from-scratch-in-pytorch/

# ResNet from scratch in PyTorch

Concept of residual connections \
Problem with going to deep: gradients from the loss function start to shrink to zero an thus the weights do not get updated (vanishing gradients). \
ResNet solution for this: skip connections that allow the gradient to be directly backpropagated to earlier layers. \
Resnet paper: https://arxiv.org/pdf/1512.03385.pdf \

In [30]:
import numpy as np
import torch 
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


## Loading the Dataset

In [39]:
def data_loader(data_dir,
                batch_size,
                random_seed=42,
                valid_size=0.1,
                shuffle=True,
                test=False):
    
    normalize = transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],std=[0.2023, 0.1994, 0.2010])
    
    # define transforms
    transform = transforms.Compose([transforms.Resize((28,28)), transforms.ToTensor(), normalize])
    
    if test:
        dataset = datasets.MNIST(root=data_dir, train=False, download=True, transform=transform)
        
        data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
        
        return data_loader
    
    # load the dataset
    
    train_dataset = datasets.MNIST(root=data_dir, train=True, download=True, transform=transform)
    valid_dataset = datasets.MNIST(root=data_dir, train=True, download=True, transform=transform)
    
    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    
    if shuffle:
        np.random.seed(42)
        np.random.shuffle(indices)
        
    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)
    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
    valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, sampler=valid_sampler)
    return (train_loader, valid_loader)

# MNIST dataset
train_loader, valid_loader = data_loader(data_dir='./data', batch_size=64)

test_loader = data_loader(data_dir='./data', batch_size=64, test=True)



TypeError: 'DataLoader' object is not subscriptable

## Residual Block


In [32]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None): # downsample is optional parameter skip connection
        super(ResidualBlock, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU())
        self.conv2 = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels))
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels = out_channels
    
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        
        if self.downsample:
            residual = self.downsample(x)
        
        out += residual
        out = self.relu(out)
        
        return out

## ResNet Model ResNet34

In [33]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        
        self.inplanes = 64 # 64 is the number of filters in the first layer
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU())
        
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1) # pooling layer
        
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2)
        
        self.avgpool = nn.AvgPool2d(7, stride=1) # pooling layer for the end of the network
        
        self.fc = nn.Linear(512, num_classes) # fully connected layer
        
    def _make_layer(self, block, planes, blocks, stride = 1):
        downsample = None
        
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes))
            
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv1(x) 
        
        out = self.maxpool(out)
        
        out = self.layer0(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        return out

In [36]:
num_classes = 10
num_epochs = 20
batch_size = 16
learning_rate = 0.01

model = ResNet(ResidualBlock, [3, 4, 6, 3], num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.001)

total_step = len(train_loader)

In [38]:
import gc
total_step = len(train_loader)


for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        print(images.shape)
        print(labels.shape)
        images = images.to(device)
        labels = labels.to(device)
        
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        del images, labels, outputs
        torch.cuda.empty_cache()
        gc.collect()
        
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

RuntimeError: output with shape [1, 28, 28] doesn't match the broadcast shape [3, 28, 28]

In [ ]:
# Validation

with torch.no_grad():
    correct = 0
    total = 0
    
    for images, labels in valid_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, prediceted = torch.max(outputs.data, 1)  # _ is max value, predicted is index
        total += labels.size(0)
        correct += (prediceted == labels).sum().item()
        del images, labels, outputs
        
    print("Accuracy of the network on the {} validation images: {} %".format(5000, 100*correct/total))

## Testing the Model

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, prediceted = torch.max(outputs.data, 1)  # _ is max value, predicted is index
        total += labels.size(0)
        correct += (prediceted == labels).sum().item()
        del images, labels, outputs
        
    print("Accuracy of the network on the {} test images: {} %".format(5000, 100*correct/total))